In [3]:
import dlib
import numpy as np
import cv2
import face_recognition as face 

# โหลดโมเดลตรวจจับใบหน้า
detector = dlib.get_frontal_face_detector()

# โหลดโมเดลรู้จำใบหน้า
recognizer = dlib.face_recognition_model_v1("dlib_face_recognition_resnet_model_v1.dat")

#ใบหน้าคนที่ต้องการรู้จำเป็นreference #คนที่1
pop_image = face.load_image_file("PK2.jpg")
pop_face_encoding = face.face_encodings(pop_image)[0]

# อ่านชุดข้อมูลใบหน้าและป้ายชื่อ
for label in os.listdir(dataset_path):
    label_path = os.path.join(dataset_path, label)
    for image_file in os.listdir(label_path):
        image_path = os.path.join(label_path, image_file)
        image = cv2.imread(image_path)
        face_images.append(image)
        face_labels.append(label)

# ทำการตรวจจับใบหน้าและสร้างเวกเตอร์รูปหน้าสำหรับแต่ละบุคคล
face_descriptors = []
for image in face_images:
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)
    for face in faces:
        shape = predictor(gray, face)
        face_descriptor = recognizer.compute_face_descriptor(image, shape)
        face_descriptors.append(face_descriptor)

# แปลงรายการ face_descriptors เป็น numpy array
face_descriptors = np.array(face_descriptors)

# บันทึกโมเดลรู้จำใบหน้า
recognizer_path = "face_recognizer.dat"
dlib.save_face_recognition_model(recognizer_path, face_descriptors, face_labels)


RuntimeError: An error occurred while trying to read the first object from the file 'dlib_face_recognition_resnet_model_v1.dat'.
ERROR: Unexpected version found while deserializing dlib::add_loss_layer.


In [1]:
import face_recognition as face 
import numpy as np 
import cv2

#ORIGINAL_CODE_CREDIT:  https://github.com/ageitgey/face_recognition/blob/master/examples/facerec_from_webcam_faster.py

#ดึงวิดีโอตัวอย่างเข้ามา, ถ้าต้องการใช้webcamให้ใส่เป็น0
video_capture = cv2.VideoCapture(0) 

#ใบหน้าคนที่ต้องการรู้จำเป็นreference #คนที่1
pun_image = face.load_image_file("PK2.jpg")
pun_face_encoding = face.face_encodings(pun_image)[0]

#ประกาศตัวแปร
face_locations = []
face_encodings = []
face_names = []
face_percent = []
#ตัวแปรนี้ใช้สำหรับคิดเฟรมเว้นเฟรมเพื่อเพิ่มfps 
process_this_frame = True

known_face_encodings = [pun_face_encoding]
known_face_names = ["PUNPK"]

#loopคำนวณแต่ละเฟรมของวิดีโอ
while True:
    #อ่านค่าแต่ละเฟรมจากวิดีโอ
    ret, frame = video_capture.read()
    if ret:
        #ลดขนาดสองเท่าเพื่อเพิ่มfps 
        small_frame = cv2.resize(frame, (0,0), fx=0.5,fy=0.5)
        #เปลี่ยน bgrเป็น rgb 
        rgb_small_frame = small_frame[:,:,::-1]

        face_names = []
        face_percent = []

        if process_this_frame:
            #ค้นหาตำแหน่งใบหน้าในเฟรม 
            face_locations = face.face_locations(rgb_small_frame, model="cnn")
            #นำใบหน้ามาหาfeaturesต่างๆที่เป็นเอกลักษณ์ 
            face_encodings = face.face_encodings(rgb_small_frame, face_locations)
            
            #เทียบแต่ละใบหน้า
            for face_encoding in face_encodings:
                face_distances = face.face_distance(known_face_encodings, face_encoding)
                best = np.argmin(face_distances)
                face_percent_value = 1-face_distances[best]

                #กรองใบหน้าที่ความมั่นใจ50% ปล.สามารถลองเปลี่ยนได้
                if face_percent_value >= 0.5:
                    name = known_face_names[best]
                    percent = round(face_percent_value*100,2)
                    face_percent.append(percent)
                else:
                    name = "UNKNOWN"
                    face_percent.append(0)
                face_names.append(name)

        #วาดกล่องและtextเมื่อแสดงผลออกมาออกมา
        for (top,right,bottom, left), name, percent in zip(face_locations, face_names, face_percent):
            top*= 2
            right*= 2
            bottom*= 2
            left*= 2

            if name == "UNKNOWN":
                color = [46,2,209]
            else:
                color = [255,102,51]

            cv2.rectangle(frame, (left,top), (right,bottom), color, 2)
            cv2.rectangle(frame, (left-1, top -30), (right+1,top), color, cv2.FILLED)
            cv2.rectangle(frame, (left-1, bottom), (right+1,bottom+30), color, cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, name, (left+6, top-6), font, 0.6, (255,255,255), 1)
            cv2.putText(frame, "MATCH: "+str(percent)+"%", (left+6, bottom+23), font, 0.6, (255,255,255), 1)


        #สลับค่าเป็นค่าตรงข้ามเพื่อให้คิดเฟรมเว้นเฟรม
        process_this_frame = not process_this_frame

        #แสดงผลลัพท์ออกมา
        cv2.imshow("Video", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    else:
        break

#ล้างค่าต่างๆเมื่อปิดโปรแกรม
video_capture.release()
cv2.destroyAllWindows()